# Introduction: Business Problem
This project is for new graduate students, personal development plan, who are interested in Data field (Data Science, Data Analyst, ...) to get to know more about the current job market and requirements. Then, they can equip themselves better to current job market without taking too much time spending on searching information in the internet

Further, the research also give benefits for the Head Hunter Companies, the Universities, and the IT centers to grab the latest information of recruitment tendency regarding Data Field in Vietnam. From then, these mentioned companies and these collegues can support further their students or applicants to enhance their knowledge, skills to adapt the latest environment

With these purposes, we try to craw data (recruitment info) from the top websites in Vietnam. Then, doing analyst (EDA) is to find out the insights on data (job titile, company, requirements, skills ..). Later, recommend system will be built to support applicants to match their current abilities with company's requirement

## Data
Based on definition of our problem, factors that will influence our decission are:
* The number of jobs posted in the recruitement website
* The Key words for searching "Data/Machine Learning/AI/Du lieu.."
* The Number of websites to get data from 

We decided to use **Exploratory Data Analyst, Text Analyst** later than applies Recommendation System to match these requirements 

Following data sources will be needed to extract/generate the required information:
* The number of jobs posted will be analysted by Title, Company, City, level
* The text analyst will be analysted to find year of experience requirements, computer languages requirements, skills and degree
* The recommendation system of 'Content Based' will be built to match the inputs 

## Data Mining 
Firstly, we need data to work on it. Below are our codings to get data as follows:
* Website name: https://careerbuilder.vn (as one of the top popular websites in Vietnam with the highest volumes of jobs of Data)
* Key words to search: Data- , Machine-Learning, AI, du-lieu
* Note: The craw data includes the wording "Data" in either Job-title or Job-description so we decided to get all first
* Tool: use BeautifulSoup to get more than 3000 jobs posted in the website

In [ ]:
#Import the library 
import json

import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import pandas as pd
import re
import string
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import requests

%pylab inline

In [2]:
# Get all data with many pages at the same time with key word being ' DATA'
d=[]
for i in range(38): # Pls calculate the number of pages in the website to input the total page for data scrawling
    url="https://careerbuilder.vn/jobs/data-k-page-" + str(i) + "-en.html" #Search for "Data *"
    print(url)
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    job_links = [link.find('a', {'class': 'job_link'}).get('href') for link in soup.find_all('div',{'class': 'job-item'})]
    #print(job_links)
    
    for url in job_links: 
        req = requests.get(url)
        soup = BeautifulSoup(req.content, 'html.parser')
        #print(url)
        if (soup.find('p',attrs={"class":"title"})) is not None:
            job_title= soup.find('p',attrs={"class":"title"}).contents[0]
        else:
            job_title = 'No Title'
        if (soup.find('a',attrs={"class":"employer job-company-name"})) is not None:
            job_company=soup.find('a',attrs={"class":"employer job-company-name"}).contents[0]
        else:
            job_company = 'No Name'
        if soup.find('div',attrs={"class":"detail-box has-background"}) is not None:
            job_industry=(soup.find('div',attrs={"class":"detail-box has-background"}).contents[1]).find('a') #remove-all
        else:
            job_industry= 'Not found'
        if soup.find('div',attrs={"class":"detail-box has-background"}) is not None:
            job_type=(soup.find('div',attrs={"class":"detail-box has-background"}).contents[1]).find_all('p')[2].contents[0]
        else:
            job_type = 'Not found'
        if (len((soup.find('div',attrs={"class":"map"}).contents))==3):
            job_location = 'Ho Chi Minh'
        elif (type((soup.find('div',attrs={"class":"map"}).contents[3]).find('a')) is int):
            job_location = 'Ho Chi Minh'
        else:
            job_location=(soup.find('div',attrs={"class":"map"}).contents[3]).find('a').contents[0]
        if len((soup.find_all('div',attrs={"class":"detail-box has-background"}))) > 0:
            job_salary=(soup.find_all('div',attrs={"class":"detail-box has-background"})[1]).find('p').contents[0]
        else:
            job_salary = 'Negotiable'
        if len(soup.find_all('div',attrs={"class":"detail-box has-background"})) > 0:
            job_experience=(soup.find_all('div',attrs={"class":"detail-box has-background"})[1]).find_all('p')[1]
        else:
            job_experience = 'fresher'
        if len(soup.find_all('div',attrs={"class":"detail-box has-background"})) > 0:
            job_level=(soup.find_all('div',attrs={"class":"detail-box has-background"})[1]).find_all('p')[2].contents[0]
        else: 
            job_level = 'Staff'
        if len(soup.find_all('ul',attrs={"class":"welfare-list"})) >0:
            job_benefit=soup.find_all('ul',attrs={"class":"welfare-list"})[0].find_all('li') #remove-all
        else:
            job_benefit = 'Not found'
        job_desc=soup.find_all('div',attrs={"class":"detail-row"})[1].find_all('p')
        job_requirements=soup.find_all('div',attrs={"class":"detail-row"})[2].find_all('p')
        d.append({'Title': job_title, 
                         'Company': job_company,
                         'Industry': job_industry,
                         'Type': job_type,
                         'Location': job_location,
                         'Salary' : job_salary,
                         'Experience': job_experience,
                         'Level': job_level,
                         'Benefit': job_benefit,
                         'Description': job_desc,
                         'Requirement': job_requirements,
                         'Link':url
             })

data=pd.DataFrame(d)

https://careerbuilder.vn/jobs/data-k-page-0-en.html
https://careerbuilder.vn/jobs/data-k-page-1-en.html
https://careerbuilder.vn/jobs/data-k-page-2-en.html
https://careerbuilder.vn/jobs/data-k-page-3-en.html
https://careerbuilder.vn/jobs/data-k-page-4-en.html
https://careerbuilder.vn/jobs/data-k-page-5-en.html
https://careerbuilder.vn/jobs/data-k-page-6-en.html
https://careerbuilder.vn/jobs/data-k-page-7-en.html
https://careerbuilder.vn/jobs/data-k-page-8-en.html
https://careerbuilder.vn/jobs/data-k-page-9-en.html
https://careerbuilder.vn/jobs/data-k-page-10-en.html
https://careerbuilder.vn/jobs/data-k-page-11-en.html
https://careerbuilder.vn/jobs/data-k-page-12-en.html
https://careerbuilder.vn/jobs/data-k-page-13-en.html
https://careerbuilder.vn/jobs/data-k-page-14-en.html
https://careerbuilder.vn/jobs/data-k-page-15-en.html
https://careerbuilder.vn/jobs/data-k-page-16-en.html
https://careerbuilder.vn/jobs/data-k-page-17-en.html
https://careerbuilder.vn/jobs/data-k-page-18-en.html
htt

In [3]:
data.count()

Title          1900
Company        1900
Industry       1900
Type           1900
Location       1900
Salary         1900
Experience     1900
Level          1900
Benefit        1900
Description    1900
Requirement    1900
Link           1900
dtype: int64

In [4]:
data.head()

,Title,Company,Industry,Type,Location,Salary,Experience,Level,Benefit,Description,Requirement,Link
0,Data Engineer (ETL/ Data Stage/ Data Warehouse),No Name,Not found,Not found,Ho Chi Minh,Negotiable,fresher,Staff,"[[[], Insurance], [[], Travel], [[], Incent...","[[[OBJECTIVE:]], [Data Warehouse Management Sp...","[[University Graduate, major: Computer Science...",https://careerbuilder.vn/en/search-job/data-en...
1,Data Analyst,Dai-ichi Life Việt Nam,[\r\n ...,Permanent,Ho Chi Minh,Competitive,[\r\n ...,Experienced (Non - Manager),"[[[], Insurance], [[], Travel], [[], Allowa...",[[- Make a data report for New Business Depart...,"[[- University degree or above (Technology, Fo...",https://careerbuilder.vn/en/search-job/data-an...
2,Data Analyst,Công Ty Cổ Phần Công Nghệ Sen Đỏ,[\r\n ...,Permanent,Ho Chi Minh,Competitive,[\r\n ...,Experienced (Non - Manager),"[[[], Insurance], [[], Healthcare], [[], An...",[],"[[Required degree/experience], [Required quali...",https://careerbuilder.vn/en/search-job/data-an...
3,Data Engineer,Công Ty Cổ Phần Quốc Tế Sơn Hà,[\r\n ...,Permanent,Ha Noi,Competitive,[\r\n ...,Experienced (Non - Manager),"[[[], Insurance], [[], Travel], [[], Travel...",[[- Tham gia xây dựng và quản trị hệ thống Big...,[[- Hiểu biết cơ bản về Hadoop eco-system. Yêu...,https://careerbuilder.vn/en/search-job/data-en...
4,Data Engineer,Công Ty CP Việt Tinh Anh,[\r\n ...,Permanent,Ho Chi Minh,Competitive,[Experienced (Non - Manager)],30/03/2021,"[[[], Insurance], [[], Incentive bonus], [[]...",[],[],https://careerbuilder.vn/en/search-job/data-en...


In [5]:
# Save under CSV file
data.to_csv("Career_Builder_data.csv")

In [6]:
# For data mining with short list of jobs- Get all data with 1 page only with key word being "Machine Learning"
d=[]
url='https://careerbuilder.vn/jobs/machine-learning-k-en.html'
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
jobs_table = soup.find_all('div',{'class': 'job-item'})
job_links = [title.find('a', {'class': 'job_link'}).get('href') for title in jobs_table]
for url in job_links: 
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    #print(url)
    if (soup.find('p',attrs={"class":"title"})) is not None:
        job_title= soup.find('p',attrs={"class":"title"}).contents[0]
    else:
        job_title = 'No Title'
    if (soup.find('a',attrs={"class":"employer job-company-name"})) is not None:
        job_company=soup.find('a',attrs={"class":"employer job-company-name"}).contents[0]
    else:
        job_company = 'No Name'
    if soup.find('div',attrs={"class":"detail-box has-background"}) is not None:
        job_industry=(soup.find('div',attrs={"class":"detail-box has-background"}).contents[1]).find('a') #remove-all
    else:
        job_industry= 'Not found'
    if soup.find('div',attrs={"class":"detail-box has-background"}) is not None:
        job_type=(soup.find('div',attrs={"class":"detail-box has-background"}).contents[1]).find_all('p')[2].contents[0]
    else:
        job_type = 'Not found'
    if (len((soup.find('div',attrs={"class":"map"}).contents))==3):
        job_location = 'Ho Chi Minh'
    elif (type((soup.find('div',attrs={"class":"map"}).contents[3]).find('a')) is int):
        job_location = 'Ho Chi Minh'
    else:
        job_location=(soup.find('div',attrs={"class":"map"}).contents[3]).find('a').contents[0]
    if len((soup.find_all('div',attrs={"class":"detail-box has-background"}))) > 0:
        job_salary=(soup.find_all('div',attrs={"class":"detail-box has-background"})[1]).find('p').contents[0]
    else:
        job_salary = 'Negotiable'
    if len(soup.find_all('div',attrs={"class":"detail-box has-background"})) > 0:
        job_experience=(soup.find_all('div',attrs={"class":"detail-box has-background"})[1]).find_all('p')[1]
    else:
        job_experience = 'fresher'
    if len(soup.find_all('div',attrs={"class":"detail-box has-background"})) > 0:
        job_level=(soup.find_all('div',attrs={"class":"detail-box has-background"})[1]).find_all('p')[2].contents[0]
    else: 
        job_level = 'Staff'
    if len(soup.find_all('ul',attrs={"class":"welfare-list"})) >0:
        job_benefit=soup.find_all('ul',attrs={"class":"welfare-list"})[0].find_all('li') #remove-all
    else:
        job_benefit = 'Not found'
    job_desc=soup.find_all('div',attrs={"class":"detail-row"})[1].find_all('p')
    job_requirements=soup.find_all('div',attrs={"class":"detail-row"})[2].find_all('p')
    d.append({'Title': job_title, 
                     'Company': job_company,
                     'Industry': job_industry,
                     'Type': job_type,
                     'Location': job_location,
                     'Salary' : job_salary,
                     'Experience': job_experience,
                     'Level': job_level,
                     'Benefit': job_benefit,
                     'Description': job_desc,
                     'Requirement': job_requirements,
                     'Link':url
         })
data=pd.DataFrame(d)

In [7]:
data.count()

Title          43
Company        43
Industry       43
Type           43
Location       43
Salary         43
Experience     43
Level          43
Benefit        43
Description    43
Requirement    43
Link           43
dtype: int64

In [8]:
data.to_csv("Career_Builder_machine_learning.csv")

In [3]:
# Get all data with many pages at the same time - Key word search ' DU LIEU'
d=[]
for i in range(41):
    url="https://careerbuilder.vn/jobs/du-lieu-k-page-" + str(i) + "-en.html" #Search for "DU LIEU *"
    print(url)
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    job_links = [link.find('a', {'class': 'job_link'}).get('href') for link in soup.find_all('div',{'class': 'job-item'})]
    #print(job_links)
    
    for url in job_links: 
        req = requests.get(url)
        soup = BeautifulSoup(req.content, 'html.parser')
        #print(url)
        if (soup.find('p',attrs={"class":"title"})) is not None:
            job_title= soup.find('p',attrs={"class":"title"}).contents[0]
        else:
            job_title = 'No Title'
        if (soup.find('a',attrs={"class":"employer job-company-name"})) is not None:
            job_company=soup.find('a',attrs={"class":"employer job-company-name"}).contents[0]
        else:
            job_company = 'No Name'
        if soup.find('div',attrs={"class":"detail-box has-background"}) is not None:
            job_industry=(soup.find('div',attrs={"class":"detail-box has-background"}).contents[1]).find('a') #remove-all
        else:
            job_industry= 'Not found'
        if soup.find('div',attrs={"class":"detail-box has-background"}) is not None:
            job_type=(soup.find('div',attrs={"class":"detail-box has-background"}).contents[1]).find_all('p')[2].contents[0]
        else:
            job_type = 'Not found'
        if (len((soup.find('div',attrs={"class":"map"}).contents))==3):
            job_location = 'Ho Chi Minh'
        elif (type((soup.find('div',attrs={"class":"map"}).contents[3]).find('a')) is int):
            job_location = 'Ho Chi Minh'
        else:
            job_location=(soup.find('div',attrs={"class":"map"}).contents[3]).find('a').contents[0]
        if len((soup.find_all('div',attrs={"class":"detail-box has-background"}))) > 0:
            job_salary=(soup.find_all('div',attrs={"class":"detail-box has-background"})[1]).find('p').contents[0]
        else:
            job_salary = 'Negotiable'
        if len(soup.find_all('div',attrs={"class":"detail-box has-background"})) > 0:
            job_experience=(soup.find_all('div',attrs={"class":"detail-box has-background"})[1]).find_all('p')[1]
        else:
            job_experience = 'fresher'
        if len(soup.find_all('div',attrs={"class":"detail-box has-background"})) > 0:
            job_level=(soup.find_all('div',attrs={"class":"detail-box has-background"})[1]).find_all('p')[2].contents[0]
        else: 
            job_level = 'Staff'
        if len(soup.find_all('ul',attrs={"class":"welfare-list"})) >0:
            job_benefit=soup.find_all('ul',attrs={"class":"welfare-list"})[0].find_all('li') #remove-all
        else:
            job_benefit = 'Not found'
        job_desc=soup.find_all('div',attrs={"class":"detail-row"})[1].find_all('p')
        job_requirements=soup.find_all('div',attrs={"class":"detail-row"})[2].find_all('p')
        d.append({'Title': job_title, 
                         'Company': job_company,
                         'Industry': job_industry,
                         'Type': job_type,
                         'Location': job_location,
                         'Salary' : job_salary,
                         'Experience': job_experience,
                         'Level': job_level,
                         'Benefit': job_benefit,
                         'Description': job_desc,
                         'Requirement': job_requirements,
                         'Link':url
             })

data=pd.DataFrame(d)

https://careerbuilder.vn/jobs/du-lieu-k-page-0-en.html
https://careerbuilder.vn/jobs/du-lieu-k-page-1-en.html
https://careerbuilder.vn/jobs/du-lieu-k-page-2-en.html
https://careerbuilder.vn/jobs/du-lieu-k-page-3-en.html
https://careerbuilder.vn/jobs/du-lieu-k-page-4-en.html
https://careerbuilder.vn/jobs/du-lieu-k-page-5-en.html
https://careerbuilder.vn/jobs/du-lieu-k-page-6-en.html
https://careerbuilder.vn/jobs/du-lieu-k-page-7-en.html
https://careerbuilder.vn/jobs/du-lieu-k-page-8-en.html
https://careerbuilder.vn/jobs/du-lieu-k-page-9-en.html
https://careerbuilder.vn/jobs/du-lieu-k-page-10-en.html
https://careerbuilder.vn/jobs/du-lieu-k-page-11-en.html
https://careerbuilder.vn/jobs/du-lieu-k-page-12-en.html
https://careerbuilder.vn/jobs/du-lieu-k-page-13-en.html
https://careerbuilder.vn/jobs/du-lieu-k-page-14-en.html
https://careerbuilder.vn/jobs/du-lieu-k-page-15-en.html
https://careerbuilder.vn/jobs/du-lieu-k-page-16-en.html
https://careerbuilder.vn/jobs/du-lieu-k-page-17-en.html
ht

In [4]:
data.count()

Title          2050
Company        2050
Industry       2050
Type           2050
Location       2050
Salary         2050
Experience     2050
Level          2050
Benefit        2050
Description    2050
Requirement    2050
Link           2050
dtype: int64

In [5]:
data.to_csv("Career_Builder_du_lieu.csv")

In [6]:
# Get all data with many pages at the same time - Key word search "AI"
d=[]
for i in range(8):
    url="https://careerbuilder.vn/jobs/AI-k-page-" + str(i) + "-en.html" #Search for "AI *"
    print(url)
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    job_links = [link.find('a', {'class': 'job_link'}).get('href') for link in soup.find_all('div',{'class': 'job-item'})]
    #print(job_links)
    
    for url in job_links: 
        req = requests.get(url)
        soup = BeautifulSoup(req.content, 'html.parser')
        #print(url)
        if (soup.find('p',attrs={"class":"title"})) is not None:
            job_title= soup.find('p',attrs={"class":"title"}).contents[0]
        else:
            job_title = 'No Title'
        if (soup.find('a',attrs={"class":"employer job-company-name"})) is not None:
            job_company=soup.find('a',attrs={"class":"employer job-company-name"}).contents[0]
        else:
            job_company = 'No Name'
        if soup.find('div',attrs={"class":"detail-box has-background"}) is not None:
            job_industry=(soup.find('div',attrs={"class":"detail-box has-background"}).contents[1]).find('a') #remove-all
        else:
            job_industry= 'Not found'
        if soup.find('div',attrs={"class":"detail-box has-background"}) is not None:
            job_type=(soup.find('div',attrs={"class":"detail-box has-background"}).contents[1]).find_all('p')[2].contents[0]
        else:
            job_type = 'Not found'
        if (len((soup.find('div',attrs={"class":"map"}).contents))==3):
            job_location = 'Ho Chi Minh'
        elif (type((soup.find('div',attrs={"class":"map"}).contents[3]).find('a')) is int):
            job_location = 'Ho Chi Minh'
        else:
            job_location=(soup.find('div',attrs={"class":"map"}).contents[3]).find('a').contents[0]
        if len((soup.find_all('div',attrs={"class":"detail-box has-background"}))) > 0:
            job_salary=(soup.find_all('div',attrs={"class":"detail-box has-background"})[1]).find('p').contents[0]
        else:
            job_salary = 'Negotiable'
        if len(soup.find_all('div',attrs={"class":"detail-box has-background"})) > 0:
            job_experience=(soup.find_all('div',attrs={"class":"detail-box has-background"})[1]).find_all('p')[1]
        else:
            job_experience = 'fresher'
        if len(soup.find_all('div',attrs={"class":"detail-box has-background"})) > 0:
            job_level=(soup.find_all('div',attrs={"class":"detail-box has-background"})[1]).find_all('p')[2].contents[0]
        else: 
            job_level = 'Staff'
        if len(soup.find_all('ul',attrs={"class":"welfare-list"})) >0:
            job_benefit=soup.find_all('ul',attrs={"class":"welfare-list"})[0].find_all('li') #remove-all
        else:
            job_benefit = ''
        job_desc=soup.find_all('div',attrs={"class":"detail-row"})[1].find_all('p')
        job_requirements=soup.find_all('div',attrs={"class":"detail-row"})[2].find_all('p')
        d.append({'Title': job_title, 
                         'Company': job_company,
                         'Industry': job_industry,
                         'Type': job_type,
                         'Location': job_location,
                         'Salary' : job_salary,
                         'Experience': job_experience,
                         'Level': job_level,
                         'Benefit': job_benefit,
                         'Description': job_desc,
                         'Requirement': job_requirements,
                         'Link':url
             })

data=pd.DataFrame(d)

https://careerbuilder.vn/jobs/AI-k-page-0-en.html
https://careerbuilder.vn/jobs/AI-k-page-1-en.html
https://careerbuilder.vn/jobs/AI-k-page-2-en.html
https://careerbuilder.vn/jobs/AI-k-page-3-en.html
https://careerbuilder.vn/jobs/AI-k-page-4-en.html
https://careerbuilder.vn/jobs/AI-k-page-5-en.html
https://careerbuilder.vn/jobs/AI-k-page-6-en.html
https://careerbuilder.vn/jobs/AI-k-page-7-en.html


In [7]:
data.count()

Title          400
Company        400
Industry       400
Type           400
Location       400
Salary         400
Experience     400
Level          400
Benefit        400
Description    400
Requirement    400
Link           400
dtype: int64

In [8]:
data.to_csv("Career_Builder_AI.csv")

In [2]:
# Get all data with many pages at the same time - Key word search "Analyst"
d=[]
for i in range(7):
    url="https://careerbuilder.vn/jobs/analyst-k-page-" + str(i) + "-en.html" 
    print(url)
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    job_links = [link.find('a', {'class': 'job_link'}).get('href') for link in soup.find_all('div',{'class': 'job-item'})]
    #print(job_links)
    
    for url in job_links: 
        req = requests.get(url)
        soup = BeautifulSoup(req.content, 'html.parser')
        #print(url)
        if (soup.find('p',attrs={"class":"title"})) is not None:
            job_title= soup.find('p',attrs={"class":"title"}).contents[0]
        else:
            job_title = 'No Title'
        if (soup.find('a',attrs={"class":"employer job-company-name"})) is not None:
            job_company=soup.find('a',attrs={"class":"employer job-company-name"}).contents[0]
        else:
            job_company = 'No Name'
        if soup.find('div',attrs={"class":"detail-box has-background"}) is not None:
            job_industry=(soup.find('div',attrs={"class":"detail-box has-background"}).contents[1]).find('a') #remove-all
        else:
            job_industry= 'Not found'
        if soup.find('div',attrs={"class":"detail-box has-background"}) is not None:
            job_type=(soup.find('div',attrs={"class":"detail-box has-background"}).contents[1]).find_all('p')[2].contents[0]
        else:
            job_type = 'Not found'
        if (len((soup.find('div',attrs={"class":"map"}).contents))==3):
            job_location = 'Ho Chi Minh'
        elif (type((soup.find('div',attrs={"class":"map"}).contents[3]).find('a')) is int):
            job_location = 'Ho Chi Minh'
        else:
            job_location=(soup.find('div',attrs={"class":"map"}).contents[3]).find('a').contents[0]
        if len((soup.find_all('div',attrs={"class":"detail-box has-background"}))) > 0:
            job_salary=(soup.find_all('div',attrs={"class":"detail-box has-background"})[1]).find('p').contents[0]
        else:
            job_salary = 'Negotiable'
        if len(soup.find_all('div',attrs={"class":"detail-box has-background"})) > 0:
            job_experience=(soup.find_all('div',attrs={"class":"detail-box has-background"})[1]).find_all('p')[1]
        else:
            job_experience = 'fresher'
        if len(soup.find_all('div',attrs={"class":"detail-box has-background"})) > 0:
            job_level=(soup.find_all('div',attrs={"class":"detail-box has-background"})[1]).find_all('p')[2].contents[0]
        else: 
            job_level = 'Staff'
        if len(soup.find_all('ul',attrs={"class":"welfare-list"})) >0:
            job_benefit=soup.find_all('ul',attrs={"class":"welfare-list"})[0].find_all('li') #remove-all
        else:
            job_benefit = ''
        job_desc=soup.find_all('div',attrs={"class":"detail-row"})[1].find_all('p')
        job_requirements=soup.find_all('div',attrs={"class":"detail-row"})[2].find_all('p')
        d.append({'Title': job_title, 
                         'Company': job_company,
                         'Industry': job_industry,
                         'Type': job_type,
                         'Location': job_location,
                         'Salary' : job_salary,
                         'Experience': job_experience,
                         'Level': job_level,
                         'Benefit': job_benefit,
                         'Description': job_desc,
                         'Requirement': job_requirements,
                         'Link':url
             })

data=pd.DataFrame(d)

https://careerbuilder.vn/jobs/analyst-k-page-0-en.html
https://careerbuilder.vn/jobs/analyst-k-page-1-en.html
https://careerbuilder.vn/jobs/analyst-k-page-2-en.html
https://careerbuilder.vn/jobs/analyst-k-page-3-en.html
https://careerbuilder.vn/jobs/analyst-k-page-4-en.html
https://careerbuilder.vn/jobs/analyst-k-page-5-en.html
https://careerbuilder.vn/jobs/analyst-k-page-6-en.html


In [3]:
data.count()

Title          350
Company        350
Industry       350
Type           350
Location       350
Salary         350
Experience     350
Level          350
Benefit        350
Description    350
Requirement    350
Link           350
dtype: int64

In [4]:
data.to_csv("Career_Builder_Analyst.csv")

## Conclustion:
#### What did I learn from this part:
* How to get data from the website (HTML/data input might empty so if..else should be used)
* Should use one language to scraw data (Vietnamese or English) 
* Try to get the core item/data (avoid invalid signs to save time for data-preprocessing) 
* Should stop 2-4 hours after scrawling data from the website for more than 3 hours continiously 

#### What I can do better?
* Think of other websites to get more data 
* Function might help reduce coding & saving time 
